<a href="https://colab.research.google.com/github/upriser72/Bridges-Distribution-Gap-in-Language-Model-Fine-Tuning/blob/main/mT5_Vanilla_Fine_Tuning_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade transformers datasets pandas torch sentencepiece accelerate

In [ ]:
import torch
from datasets import Dataset
import pandas as pd
from transformers import (
    MT5ForConditionalGeneration,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

def main():
    """
    Main function to run the fine-tuning process.
    """
    # --- 1. Load and Prepare the Dataset ---
    # Load the dataset from the CSV file.
    # Using engine='python' to handle potential parsing errors in large CSV files.
    # on_bad_lines='skip' will skip rows that have formatting issues.
    try:
        df = pd.read_csv("healthcare_dataset.csv", engine='python', on_bad_lines='skip')
    except FileNotFoundError:
        print("Error: 'healthcare_dataset.csv' not found.")
        print("Please make sure the dataset file is uploaded to your Colab environment and the name matches exactly.")
        return

    # Convert the pandas DataFrame to a Hugging Face Dataset
    dataset = Dataset.from_pandas(df)

    # --- 2. Load Tokenizer and Model ---
    model_name = "google/mt5-small"
    # The tokenizer is responsible for converting text into a format the model can understand.
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # The model is the pre-trained mT5-small architecture.
    model = MT5ForConditionalGeneration.from_pretrained(model_name)

    # --- 3. Preprocess the Data ---
    # We need to format the input and output correctly for the T5 model.
    # The model will be trained to generate the 'output' from the 'input' column.
    prefix = "diagnose: "
    max_input_length = 512
    max_target_length = 256

    def preprocess_function(examples):
        """Tokenizes the dataset."""
        # Create the combined input text for each item in the batch
        inputs = [prefix + str(inp) for inp in examples["input"]]

        # Tokenize the inputs
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

        # Tokenize the targets (outputs)
        # The 'with tokenizer.as_target_tokenizer():' block is essential for T5-style models.
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples["output"], max_length=max_target_length, truncation=True, padding="max_length")

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Apply the preprocessing function to the entire dataset
    tokenized_dataset = dataset.map(
        preprocess_function,
        batched=True,
    )

    # --- 4. Set Up Training ---
    # Define training arguments. These control various aspects of the training process.
    training_args = Seq2SeqTrainingArguments(
        output_dir="./results_mt5_medical",    # Directory to save the model and results
        num_train_epochs=50,                   # Total number of training epochs (increase for better performance on small datasets)
        per_device_train_batch_size=2,         # Batch size per device during training
        per_device_eval_batch_size=2,          # Batch size for evaluation (if used)
        warmup_steps=50,                       # Number of warmup steps for learning rate scheduler
        weight_decay=0.01,                     # Strength of weight decay
        logging_dir='./logs',                  # Directory for storing logs
        logging_steps=10,
        save_total_limit=2,                    # Only keep the last 2 saved models
        predict_with_generate=True,            # Whether to use generate to calculate generative metrics
        report_to="none",                      # Disable integration with Weights & Biases
    )

    # Data collator prepares batches of data for the model.
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # The Trainer class handles the training and evaluation loop.
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # --- 5. Start Fine-Tuning ---
    print("Starting the fine-tuning process...")
    trainer.train()
    print("Fine-tuning complete.")

    # --- 6. Save the Fine-Tuned Model ---
    final_model_path = "./fine_tuned_mt5_medical"
    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    print(f"Model saved to {final_model_path}")

    # --- 7. Inference Example ---
    print("\n--- Running Inference with the Fine-Tuned Model ---")

    # Load the fine-tuned model and tokenizer
    trained_model = MT5ForConditionalGeneration.from_pretrained(final_model_path)
    trained_tokenizer = AutoTokenizer.from_pretrained(final_model_path)

    # Define a new patient query
    patient_input = "I have a persistent dry cough for the last 3 days and a slight fever. I feel tired and have a mild headache. What could this be?"

    # Prepare the input for the model
    prompt = f"diagnose: {patient_input}"
    inputs = trained_tokenizer(prompt, return_tensors="pt").input_ids

    # Generate the output
    print("Generating diagnosis...")
    outputs = trained_model.generate(
        inputs,
        max_length=200,
        num_beams=5,
        early_stopping=True
    )

    # Decode and print the result
    generated_text = trained_tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\nPatient Query:")
    print(patient_input)
    print("\nGenerated Medical Advice:")
    print(generated_text)


if __name__ == "__main__":
    main()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pytho

Map:   0%|          | 0/112165 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/tmp/ipython-input-892899370.py:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Starting the fine-tuning process...


Step,Training Loss
10,37.705500
20,38.243500
30,34.302800
40,34.662800
50,33.503100
60,32.513100
70,29.491300
80,29.870600
90,26.416700
100,27.780800
